In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN

# ----- 常量 -----
R, L, theta, alpha = 0.04, 0.125, np.deg2rad(30), 0

In [4]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv('sorted_data.csv')

# 筛选出 Type 为 4 的数据
df_filtered = df[df['Type'] == 5]

# 显示筛选后的数据的前几行
print(df_filtered.head())


         I1        I2        I3        gx        gy         gz        ax  \
0  0.288461  0.645834  0.722757  0.833750  0.100000   1.269375  0.008530   
1  0.294017  0.441026  0.410256  0.478667 -1.110000  16.678667 -0.010335   
2  0.311111  0.422222  0.357265  0.569333 -0.880667  19.226000  0.000911   
3  0.458120  0.352137  0.215384  0.297333 -0.950000  17.830000  0.005013   
4  0.283761  0.442735  0.547009  0.048667 -2.122667  35.428000  0.012598   

         ay        az    V1real    V2real    V3real    N1    N2    N3  Type  
0  0.008545  0.012848 -113.0625 -322.3125  453.9375 -1888 -5619  7840     5  
1 -0.009001 -0.002572  154.8000  151.2000  156.6000  2560  2359  2423     5  
2  0.001774  0.003825  158.4000  158.4000  158.4000  2589  2494  2545     5  
3  0.003369  0.001497  144.0000  162.0000  167.4000  2585  2767  2902     5  
4 -0.027734 -0.002148  345.6000  331.2000  343.8000  5074  4828  4795     5  


In [6]:
import os
import pathlib

os.environ['JOBLIB_TEMP_FOLDER'] = r'E:\joblib_tmp'
pathlib.Path(r'E:\joblib_tmp').mkdir(parents=True, exist_ok=True)


from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(5,), (10,), (15,),
                           (20,), (20,10), (30,15)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['lbfgs', 'adam'],
    'max_iter': [500, 1000, 1000],
    'random_state': [0, 21, 42],
}

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

gs = GridSearchCV(
        estimator = MLPClassifier(),
        param_grid = param_grid,
        cv = cv,
        scoring = 'f1',
        n_jobs = 1,        # <- 为避免 UnicodeEncodeError
        verbose = 2        # 看进度
)
gs.fit(X_base, y)

print('Лучшие параметры:', gs.best_params_)
print('Лучший F1 (CV):',  round(gs.best_score_, 3))
best_model = gs.best_estimator_

## 3. Обучение сети на сортированных данных

In [10]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# 读取排序好的数据
sorted_data = pd.read_csv('sorted_data.csv')

# 提取特征和目标变量
# 假设特征列为 'I1', 'I2', ..., 'In'（根据实际特征列名进行调整）
X_sorted = sorted_data.drop(columns=['Type'])  # 去掉目标列
y_sorted = sorted_data['Type']

# 过滤出目标为 5 的数据
X_sorted = X_sorted[y_sorted == 5]
y_sorted = y_sorted[y_sorted == 5]

# 定义超参数网格（根据实际情况调整）
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive']
}

# 进行网格搜索
gs_sort = GridSearchCV(
    MLPClassifier(), param_grid, cv=5, scoring='f1', n_jobs=1, verbose=2
)
gs_sort.fit(X_sorted, y_sorted)

# 输出最佳结果
print('Sorted best F1:', gs_sort.best_score_.round(3))
print(gs_sort.best_params_)
best_sort = gs_sort.best_estimator_


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END activation=tanh, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s


c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\HUAWEI\AppData\Local\Pro

[CV] END activation=tanh, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=tanh, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s


c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\HUAWEI\AppData\Local\Pro

[CV] END activation=tanh, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=tanh, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s


...

c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    resul

[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu

c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    resul

[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(150,), learning_rate=adaptive, solver=sgd; total time=   0.0s
[CV] END activation=rel

c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'activation': 'tanh', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'solver': 'adam'}

## 4. Обучение сети на нормализованных данных

In [13]:
# ── 4.1  Масштабирование ───────────────────────────────────────
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X_sorted)

# ── 4.2  GridSearch ────────────────────────────────────────────
gs_norm = GridSearchCV(
    MLPClassifier(), param_grid, cv=cv, scoring='f1', n_jobs=1, verbose=2
)
gs_norm.fit(X_norm, y_sorted)

print('Norm best F1:', gs_norm.best_score_.round(3))
print(gs_norm.best_params_)
best_norm = gs_norm.best_estimator_

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(50,), learning_rate=constant, solver=adam; total time=   0.0s


c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.0s
[CV] END activation=relu, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam; total time=   0.0s
Norm best F1: 0.0
{'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'solver': 'adam'}


c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
